# 파이프라인 게시

[이전 랩](labdocs/Lab06A.md)에서는 파이프라인을 만들었습니다. 이제 해당 파이프라인을 서비스로 게시하겠습니다.

## 작업 영역에 연결

가장 먼저 해야 하는 작업은 Azure ML SDK를 사용하여 작업 영역에 연결하는 것입니다.

> **참고**: 이전 연습을 완료한 후 Azure 구독으로 인증된 세션이 만료된 경우 다시 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 파이프라인 게시

작성하여 테스트한 파이프라인은 REST 서비스로 게시할 수 있습니다. [이전 랩](labdocs/Lab06A.md)에서 파이프라인을 실행했으므로 해당 실행에 대한 참조를 가져온 다음 파이프라인을 게시하는 데 사용할 수 있습니다.

In [ ]:
# 파이프라인의 최신 실행 가져오기
experiment_name = 'diabetes-training-pipeline'
pipeline_experiment = ws.experiments.get(experiment_name)
pipeline_run = list(pipeline_experiment.get_runs())[0]

# 실행에서 파이프라인 게시
published_pipeline = pipeline_run.publish_pipeline(
    name="Diabetes_Training_Pipeline", description="Trains diabetes model", version="1.0")

published_pipeline

게시된 파이프라인에는 엔드포인트가 있습니다. [Azure Machine Learning Studio](https://ml.azure.com)의 **엔드포인트** 페이지 **파이프라인 엔드포인트** 탭에서 이 엔드포인트를 확인할 수 있습니다. 게시된 파이프라인 개체의 속성으로 엔드포인트의 URI를 확인할 수도 있습니다.

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

## 파이프라인 엔드포인트 사용

엔드포인트를 사용하려면 클라이언트 애플리케이션이 HTTP를 통해 REST 호출을 수행해야 합니다. 이 요청은 인증해야 하므로 인증 헤더가 필요합니다. 실제 애플리케이션에는 인증에 사용할 서비스 주체가 필요합니다. 여기서는 이 인증 과정을 테스트하기 위해 현재 Azure 작업 영역에 설정되어 있는 연결의 인증 헤더를 사용합니다. 다음 코드를 사용하면 해당 인증 헤더를 가져올 수 있습니다.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print("Authentication header ready.")

이제 REST 인터페이스를 호출할 수 있습니다. 파이프라인은 비동기식으로 실행되므로 식별자를 다시 가져와야 합니다. 이 식별자는 실행 중인 파이프라인 실험을 추적하는 데 사용할 수 있습니다.

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

실행 ID를 가져온 후에는 **RunDetails** 위젯을 사용하여 실행 중인 실험을 확인할 수 있습니다.

> **참고**: 각 단계는 출력을 재사용할 수 있도록 구성되었으므로 파이프라인 실행은 빠르게 완료됩니다. 이 구성은 기본적으로 과정 진행 시간을 절약하기 위해 편의상 적용된 것입니다. 실제로는 데이터가 변경될 때마다 첫 단계를 실행하고 1단계의 출력이 변경되는 경우에만 후속 단계를 트리거할 수 있습니다.

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
RunDetails(published_pipeline_run).show()

## 파이프라인 예약

당뇨병 환자를 진료하는 병원에서 매주 새 데이터를 수집하여 데이터 세트에 추가한다고 가정해 보겠습니다. 이 경우 매주 파이프라인을 실행하여 새 데이터로 모델을 다시 학습시킬 수 있습니다.

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# 매주 월요일 자정(UTC)에 파이프라인 제출
recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")
weekly_schedule = Schedule.create(ws, name="weekly-diabetes-training", 
                                  description="Based on time",
                                  pipeline_id=published_pipeline.id, 
                                  experiment_name=experiment_name, 
                                  recurrence=recurrence)
print('Pipeline scheduled.')

다음과 같이 작업 영역에 정의된 일정을 검색할 수 있습니다.

In [ ]:
schedules = Schedule.list(ws)
schedules

다음과 같이 최신 실행을 확인할 수 있습니다.

In [ ]:
pipeline_experiment = ws.experiments.get(experiment_name)
latest_run = list(pipeline_experiment.get_runs())[0]

latest_run.get_details()

> **추가 정보**: 파이프라인 예약에 대한 자세한 내용은 [설명서](https://docs.microsoft.com/azure/machine-learning/how-to-schedule-pipelines)에서 확인할 수 있습니다.